# Intro to gragh convolutional surrogate models
Eamon Whalen

In [1]:
import sys
import numpy as np
import pandas as pd
import altair as alt

sys.path.append('./models')
from feastnetSurrogateModel import FeaStNet

sys.path.append('./readers')
from loadGhGraphs import loadGhGraphs

sys.path.append('./visualization')
from altTrussViz import plotTruss, interactiveErrorPlot

sys.path.append('./util')
from gcnSurrogateUtil import *

## 1. Load simulation data

In [2]:
doeFile = "/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/design_7_N_1000.csv"
allGraphsUnfiltered = loadGhGraphs(doeFile, NUM_DV=5)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.199516
std,2.524026
min,0.006570
25%,0.015425
50%,0.024334
75%,0.046981
max,67.325867


In [3]:
plotTruss(allGraphsUnfiltered[0], showDeformed=True, defScale=100)

alt.LayerChart(...)

## 2. Filter and partition

In [4]:
allGraphs = filterbyDisp(allGraphsUnfiltered, 0.9)
trainData, valData, testData = partitionGraphList(allGraphs)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphs]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,900.000000
mean,0.028952
std,0.019071
min,0.006570
25%,0.014951
50%,0.021719
75%,0.038367
max,0.097861


## 3. Train a GCN

In [5]:
gcn = FeaStNet()
history = gcn.trainModel(trainData, valData, 
                     epochs=100, 
                     batch_size=256, 
                     flatten=True, 
                     logTrans=False, 
                     ssTrans=True, 
                     saveDir='./results/gcn01/')

plotHistory(history)

epoch: 0   trainLoss: 1.0074e+00   valLoss:9.7767e-01  time: 1.35e+00
epoch: 1   trainLoss: 9.4559e-01   valLoss:9.6948e-01  time: 1.06e+00
epoch: 2   trainLoss: 8.7655e-01   valLoss:9.5443e-01  time: 1.02e+00
epoch: 3   trainLoss: 8.6572e-01   valLoss:9.5230e-01  time: 1.01e+00
epoch: 4   trainLoss: 8.2652e-01   valLoss:9.6443e-01  time: 1.02e+00
epoch: 5   trainLoss: 7.8132e-01   valLoss:9.7489e-01  time: 1.01e+00
epoch: 6   trainLoss: 7.6093e-01   valLoss:9.2128e-01  time: 1.02e+00
epoch: 7   trainLoss: 7.3739e-01   valLoss:8.3740e-01  time: 1.01e+00
epoch: 8   trainLoss: 6.6487e-01   valLoss:7.2547e-01  time: 1.02e+00
epoch: 9   trainLoss: 6.6312e-01   valLoss:6.6179e-01  time: 1.03e+00
epoch: 10   trainLoss: 6.2967e-01   valLoss:6.2996e-01  time: 1.04e+00
epoch: 11   trainLoss: 6.1476e-01   valLoss:6.0924e-01  time: 1.03e+00
epoch: 12   trainLoss: 6.2344e-01   valLoss:5.9347e-01  time: 1.02e+00
epoch: 13   trainLoss: 5.6140e-01   valLoss:5.7514e-01  time: 1.03e+00
epoch: 14   trai

alt.Chart(...)

## 4. Test the GCN

In [6]:
trainRes = gcn.testModel(trainData)
testRes = gcn.testModel(testData)
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000016,0.002653,0.120624,0.894727,0.955943,0.494134,-5.863912
test,0.000024,0.002914,0.129688,0.758012,0.890755,0.433225,-5.498462


## 5. Visualize some predictions

In [16]:
i = 9
pred = gcn.predict([testData[i]])[0]
plotTruss(testData[i], showDeformed=True, defScale=100, prediction=pred)

alt.LayerChart(...)

In [17]:
# interactive scatter plot
alt.data_transformers.enable('json')
allPreds = gcn.predict(testData)
display(interactiveErrorPlot(testData, allPreds))
alt.data_transformers.enable('default');

alt.HConcatChart(...)